# AwkwardNN Under the Hood

- In this notebook, I explain how AwkwardNN processes the fields
from a given root file and yaml file. I will not how show much
python code here, that can be seen in the python files.

    - Note: all the field names in the yaml file must correspond
    to field names in the root file

- To be concrete, I will look specifically at some of the fields in
`test_ttbar_1000.root`

## Overview of AwkwardNN Structure

- An AwkwardNN subblock is a network block with four different
types of subnetworks (each corresponding to a different data
structure interpretation from `uproot`):
    - a fixed network
    - a jagged network
    - an object network
    - a nested network

The yaml file is essentially one big AwkwardNN subblock


In [3]:
import uproot
from awkwardNN.utils.dataset_utils_uproot import get_events_from_tree

tree = uproot.open("../data/test_ttbar_1000.root")['Delphes']

## Fixed Network Block

- Possible modes: `mlp` or `deepset`

- Takes as input arrays of fixed length, data structures interpreted
as fixed by `uproot`

- Feeds forward like a normal neural network or deepset model

- this deepset model consists of 2 networks:
    - the first network takes each individual element of an array
    as its input
    - the second network takes, as it input, an array whose elements
    are the element-wise sum of the outputs from the first network

- The output is an array whose length is specified by `embed_dim` key
in the yaml file

In [7]:
# Example of fixed data

fixed_fields = ['Jet_size', 'Particle_size', 'Muon_size', 'Event_size']
fixed_data = get_events_from_tree(tree, fixed_fields)

for i in range(5):
    print("Event {}: {}".format(i+1, fixed_data[i]))


Event 1: [[  4. 737.   0.   1.]]
Event 2: [[  2. 682.   0.   1.]]
Event 3: [[4.000e+00 1.267e+03 0.000e+00 1.000e+00]]
Event 4: [[4.00e+00 1.39e+03 0.00e+00 1.00e+00]]
Event 5: [[  3. 792.   0.   1.]]


## Jagged Network Block

- Possible modes: `lstm`, `gru`, `vanilla_rnn`, or `deepset`

- Takes as input arrays of data structures interpreted as jagged
by `uproot`

- Feeds forward like a recurrent neural network or deepset model

- this deepset model consists of 2 networks:
    - the first network takes arrays of fixed length as its input
    - the second network takes the element-wise sum of all the
    outputs from the first network as its input

- The output is an array whose length is specified by `embed_dim` key
in the yaml file. If the mode is an RNN-style network, then `embed_dim`
also corresponds to size of the _hidden-state_ for the RNN.


In [14]:
# Example of jagged data

jagged_fields = ['Jet.Mass', 'Jet.Eta', 'Jet.Phi']
jagged_data = get_events_from_tree(tree, jagged_fields)

for i in range(5):
    print("Event {}: {}\n".format(i+1, jagged_data[i]))

Event 1: [[206.97768      0.27329388   0.88772887]
 [179.3755      -0.33246595  -2.410344  ]
 [ 10.840576     0.49002475  -0.22583866]
 [ 14.236657     3.0309205   -1.910816  ]]

Event 2: [[ 1.8059798e+02 -1.5793906e-01  1.7673397e+00]
 [ 1.3346812e+02  4.8330280e-01 -1.3502347e+00]]

Event 3: [[ 2.1601949e+02  7.0078456e-01  3.0060477e+00]
 [ 1.6780676e+02  3.7452587e-01 -1.4007255e-01]
 [ 8.5620213e+00  3.4654589e+00 -2.3441007e+00]
 [ 7.6187215e+00  1.8378289e-01 -1.2875067e+00]]

Event 4: [[ 1.9607657e+02  2.4927530e-01  2.6330438e+00]
 [ 1.6945067e+02 -4.2195076e-01 -2.9567575e-01]
 [ 6.0668121e+01 -1.1219054e-01 -2.1356351e+00]
 [ 1.4513953e+01  2.9882538e+00 -9.5511639e-01]]

Event 5: [[185.33188     -0.9398658    1.6659472 ]
 [215.28633     -0.81829405  -1.5329396 ]
 [ 13.394765     2.9998572    0.36809367]]



## Object Network Block

- Possible modes: `lstm`, `gru`, `vanilla_rnn`, or `deepset`

- Takes as input arrays of data structures interpreted as object
by `uproot`

- Distinct from jagged arrays in that not only does each event
have a varying number of say jets, but each jet also has a
varying number of say particles or constituents. So it is variable on
two levels, whereas jagged arrays are variable on one level.

- Ad-hoc: feeds forward like a modified recurrent neural network
or deepset model

- Modified RNN consists of 2 RNNs stacked on top of each other:
    - the first RNN takes as input each subevent (e.g. jet or particle
    in an event) and sequentially processes it one element at a time.
    It returns a hidden state for each subevent.
    - the second RNN takes as input the all the hidden states from
    the first RNN, processing them sequentially one at a time.
    Its output is another hidden state.

- this deepset model consists of 2 standard deepset models stacked
on top of each other:
    - the first network takes each subevent, feeds forward through
    its phi network one element at time, element-wise sums the
    outputs from the phi network, and then feeds forward through
    its rho network.
    - the second network takes the outputs from each subevent
    as its inputs, passes each one through its phi network, sums
    the latent state for each subevent, then feeds forward through
    its rho network.

In [17]:
# Example of object data

object_fields = ['Jet.Constituents', 'Jet.Particles']
object_data = get_events_from_tree(tree, object_fields)

for i in range(5):
    print("Event {}".format(i+1))
    for obj_array in object_data[i]:
        print("{} {}".format(len(obj_array), list(obj_array)))
    print()


/Users/edisonweik/.pyenv/versions/3.6.11/lib/python3.6/site-packages/awkward/array/base.py:394: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return cls.numpy.array(value, copy=False)


Event 1
107 [array([[1398.]], dtype=float32), array([[1428.]], dtype=float32), array([[1275.]], dtype=float32), array([[1427.]], dtype=float32), array([[1413.]], dtype=float32), array([[1268.]], dtype=float32), array([[1452.]], dtype=float32), array([[1260.]], dtype=float32), array([[1443.]], dtype=float32), array([[1441.]], dtype=float32), array([[1448.]], dtype=float32), array([[1246.]], dtype=float32), array([[1442.]], dtype=float32), array([[1250.]], dtype=float32), array([[1439.]], dtype=float32), array([[1446.]], dtype=float32), array([[1254.]], dtype=float32), array([[1444.]], dtype=float32), array([[1445.]], dtype=float32), array([[1415.]], dtype=float32), array([[1421.]], dtype=float32), array([[1227.]], dtype=float32), array([[1230.]], dtype=float32), array([[1419.]], dtype=float32), array([[1417.]], dtype=float32), array([[1430.]], dtype=float32), array([[1393.]], dtype=float32), array([[1207.]], dtype=float32), array([[1396.]], dtype=float32), array([[1391.]], dtype=float32

## Nested Network Block

- Possible modes: `lstm`, `gru`, `vanilla_rnn`, `mlp`, or `deepset`
    - Note: the `deepset` mode here is the same one used in the
    jagged block

- Takes as input the output arrays of AwkwardNN subblocks because
by construction, a field from a rootfile can only be in a nested
block if it has its own tree structure

- The input size and how it feeds forward depends on the mode

- If the mode is `lstm`, `gru`, `vanilla_rnn`, or `deepset`
    - the input size is the output size (i.e. the `embed_dim` field)
    from its AwkwardNN subblocks. Note: all of these `embed_dim` fields
    must have the same value, or else an error will be raised.

    - the nested block feeds forward by sequentially processing
    its AwkwardNN subblocks one at a time. It then returns its hidden
    state as its output for the next network.

    - Note: it is sometimes the case that different subevents
    are null within an event. E.g. an event may have 5 jets but 0 muons.
    In this case, the AwkwardNN subblock associated with `Muon` would
    return an empty array that would be skipped over by the nested
    block.

- If the mode is `mlp`
    - the inputs size is the sum of the output sizes (i.e. the `embed_dim`
    field) from its AwkwardNN subblocks. Note: unlike the rnn-style mode,
    these `embed_dim` fields are not required to have the same value

    - the nested block feeds forward the concatenated output arrays
    from all of its AwkwardNN subblocks

    - Note: In the case that one subevent is null within an event, e.g.
    an event has 5 jets but 0 muons, the resulting input array to
    the nested block is zero padded to replace what would have
    been the output array from the muon block.


## AwkwardNN Block

- Possible modes: `mlp` or `deepset`

- Takes as input arrays of fixed length - the concatenated output
of all of its subblocks.

    - Note: not every type of subblock may be present. If the
    list of fields doesn't have a type of data structure present,
    then the network subblock associated with that data structure will
    not be in the AwkwardNN block.

- Feeds forward like a normal neural network or deepset model
    - Note: the `deepset` mode here is the same one used in the
    fixed block

